In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd  # 데이터 처리를 위한 pandas
import numpy as np  # 수치 계산을 위한 numpy
from sklearn.model_selection import train_test_split, cross_val_score  # 데이터 분할 및 교차 검증
from sklearn.impute import SimpleImputer  # 결측치 처리
from sklearn.preprocessing import OneHotEncoder, StandardScaler  # 범주형 변수 인코딩 및 수치형 변수 스케일링
from sklearn.compose import ColumnTransformer  # 컬럼별 전처리 파이프라인 구성
from sklearn.pipeline import Pipeline  # 전체 전처리 및 모델링 파이프라인 구성
from sklearn.linear_model import Ridge, Lasso, ElasticNet  # 선형 회귀 모델
from sklearn.metrics import mean_squared_error, make_scorer  # 모델 평가 지표
from sklearn.model_selection import RandomizedSearchCV  # 랜덤 서치를 통한 하이퍼파라미터 튜닝
from skopt import BayesSearchCV  # 베이지안 최적화를 통한 하이퍼파라미터 튜닝
from catboost import CatBoostRegressor
import xgboost as xgb  # XGBoost 모델
import lightgbm as lgb  # LightGBM 모델
import warnings  # 경고 메시지 처리
import joblib  # 모델 저장 및 로드
import os  # 파일 시스템 작업
import json  # JSON 파일 처리

# LightGBM의 불필요한 경고 메시지 무시 설정
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')

# 데이터 불러오기
train = pd.read_csv("../data/train.csv")

# 컬럼 선택
drop_cols = ['id', 'Calories']
numeric_feats = train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_feats = train.select_dtypes(include=['object']).columns.tolist()

# drop_cols 제외
numeric_feats = [col for col in numeric_feats if col not in drop_cols]
categorical_feats = [col for col in categorical_feats if col not in drop_cols]
main_features = numeric_feats + categorical_feats

# 수치형 변수 전처리 파이프라인 구성
numeric_transformer = Pipeline([
    ('scaler', StandardScaler())
])

# 범주형 변수 전처리 파이프라인 구성
categorical_transformer = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# 전처리 파이프라인 통합
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_feats),
    ('cat', categorical_transformer, categorical_feats)
])

X = train[main_features]  # 특성 데이터
y = train['Calories']  # 타겟 변수

# 2. 모델 후보군 정의
models = {
    'Ridge': Ridge(),  # 릿지 회귀
    'Lasso': Lasso(),  # 라쏘 회귀
    'ElasticNet': ElasticNet(),  # 엘라스틱넷 회귀
    'XGBoost': xgb.XGBRegressor(tree_method='hist', random_state=42),  # XGBoost
    'LightGBM': lgb.LGBMRegressor(random_state=42, verbose=-1),  # LightGBM
    'CatBoost': CatBoostRegressor(verbose=0, random_state=42)  # CatBoost
}

# 3. 모델 학습 및 평가 수행
results = {}  # 결과 저장 딕셔너리
for name, model in models.items():
    print(f'\n==== {name} ====')
    try:
        # 3.1 전체 파이프라인 구성 (전처리 + 모델)
        pipe = Pipeline([
            ('preprocessor', preprocessor),  # 전처리 단계
            ('reg', model)  # 모델 단계
        ])
        
        # 3.2 모델 학습
        pipe.fit(X, y)
        
        # 3.3 예측 및 성능 평가
        y_pred = pipe.predict(X)  # 예측값 생성
        rmse = np.sqrt(mean_squared_error(y, y_pred))  # RMSE 계산
        print(f'RMSE: {rmse:.4f}')
        
        results[name] = rmse  # 결과 저장
        
    except Exception as e:
        print(f"Error: {str(e)}")

# 4. 최종 결과 비교 및 정렬
print("\n==== Final Results ====")
results_df = pd.DataFrame(results.items(), columns=['Model', 'RMSE'])  # 결과 데이터프레임 생성
results_df = results_df.sort_values('RMSE')  # RMSE 기준 오름차순 정렬
print(results_df)

# 데이터 수집

In [61]:
import pandas as pd

train = pd.read_csv("../data/train.csv")
train.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


## 데이터 확인

In [62]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   id          750000 non-null  int64  
 1   Sex         750000 non-null  object 
 2   Age         750000 non-null  int64  
 3   Height      750000 non-null  float64
 4   Weight      750000 non-null  float64
 5   Duration    750000 non-null  float64
 6   Heart_Rate  750000 non-null  float64
 7   Body_Temp   750000 non-null  float64
 8   Calories    750000 non-null  float64
dtypes: float64(6), int64(2), object(1)
memory usage: 51.5+ MB


# EDA

## 결측치 확인

In [63]:
train.isnull().sum()

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64

## 수치형 변수 구분

In [75]:
numeric_feats = train.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_feats

['id',
 'Age',
 'Height',
 'Weight',
 'Duration',
 'Heart_Rate',
 'Body_Temp',
 'Calories']

## 범주형 변수 구분

In [76]:
categorical_feats = train.select_dtypes(include=['object']).columns.tolist()
categorical_feats

['Sex']

# 데이터 전처리

## 불필요한 변수 제거

In [77]:
drop_cols = ['id', 'Calories']
drop_cols

['id', 'Calories']

# 피처 엔지니어링

## 최종 주요 변수 추출

In [78]:
main_features = list(set(numeric_feats + categorical_feats) - set(drop_cols))
main_features

['Body_Temp', 'Height', 'Heart_Rate', 'Sex', 'Weight', 'Duration', 'Age']

In [79]:
X = train[main_features]
y = train['Calories']

X.shape, y.shape

((750000, 7), (750000,))

In [80]:
numeric_feats = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_feats = X.select_dtypes(include=['object']).columns.tolist()

In [82]:
numeric_transformer = Pipeline([
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_feats),
    ('cat', categorical_transformer, categorical_feats)
])

In [83]:
# 2. 모델 후보군 정의
models = {
    'Ridge': Ridge(),  # 릿지 회귀
    'Lasso': Lasso(),  # 라쏘 회귀
    'ElasticNet': ElasticNet(),  # 엘라스틱넷 회귀
    'XGBoost': xgb.XGBRegressor(tree_method='hist', random_state=42),  # XGBoost
    'LightGBM': lgb.LGBMRegressor(random_state=42, verbose=-1),  # LightGBM
    'CatBoost': CatBoostRegressor(verbose=0, random_state=42)  # CatBoost
}

# 3. 모델 학습 및 평가 수행
X = train[main_features]  # 특성 데이터
y = train['Calories']  # 타겟 변수

results = {}  # 결과 저장 딕셔너리
for name, model in models.items():
    print(f'\n==== {name} ====')
    try:
        # 3.1 전체 파이프라인 구성 (전처리 + 모델)
        pipe = Pipeline([
            ('preprocessor', preprocessor),  # 전처리 단계
            ('reg', model)  # 모델 단계
        ])
        
        # 3.2 모델 학습
        pipe.fit(X, y)
        
        # 3.3 예측 및 성능 평가
        y_pred = pipe.predict(X)  # 예측값 생성
        rmse = np.sqrt(mean_squared_error(y, y_pred))  # RMSE 계산
        print(f'RMSE: {rmse:.4f}')
        
        results[name] = rmse  # 결과 저장
        
    except Exception as e:
        print(f"Error: {str(e)}")

# 4. 최종 결과 비교 및 정렬
print("\n==== Final Results ====")
results_df = pd.DataFrame(results.items(), columns=['Model', 'RMSE'])  # 결과 데이터프레임 생성
results_df = results_df.sort_values('RMSE')  # RMSE 기준 오름차순 정렬
print(results_df)


==== Ridge ====
RMSE: 11.0974

==== Lasso ====
RMSE: 12.1011

==== ElasticNet ====
RMSE: 19.4973

==== XGBoost ====
RMSE: 3.5798

==== LightGBM ====


C:\Users\Admin\Desktop\ai_statistics\playground-series-s5e5\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


RMSE: 3.7663

==== CatBoost ====
RMSE: 3.3546

==== Final Results ====
        Model       RMSE
5    CatBoost   3.354645
3     XGBoost   3.579833
4    LightGBM   3.766349
0       Ridge  11.097400
1       Lasso  12.101098
2  ElasticNet  19.497292


In [85]:
# 3. 모델 학습 및 평가 수행
X = train[main_features]  # 특성 데이터
y = train['Calories']  # 타겟 변수

# 전체 파이프라인 구성 (전처리 + 모델)
pipe = Pipeline([
    ('preprocessor', preprocessor),  # 전처리 단계
    ('reg', CatBoostRegressor(verbose=0, random_state=42))  # 모델 단계
])

# 3.2 모델 학습
pipe.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Body_Temp', 'Height',
                                                   'Heart_Rate', 'Weight',
                                                   'Duration', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex'])])),
                ('reg',
                 <catboost.core.CatBoostRegressor object at 0x000001B2C25D20D0>)])